<a href="https://colab.research.google.com/github/AbdoulayeDiop/ner-with-bert/blob/main/ner_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d653e15ae3fb34da3d1d85fc9bb5cb8a15cae93ee1f11ba7239f9626b0d42682
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstal

In [13]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)
dataset['train'] = dataset['train'].shuffle(seed=42).select(range(1000))

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [15]:
dataset["train"][0]

{'id': '1469',
 'tokens': ['"',
  'Neither',
  'the',
  'National',
  'Socialists',
  '(',
  'Nazis',
  ')',
  'nor',
  'the',
  'communists',
  'dared',
  'to',
  'kidnap',
  'an',
  'American',
  'citizen',
  ',',
  '"',
  'he',
  'shouted',
  ',',
  'in',
  'an',
  'oblique',
  'reference',
  'to',
  'his',
  'extradition',
  'to',
  'Germany',
  'from',
  'Denmark',
  '.',
  '"'],
 'pos_tags': [0,
  12,
  12,
  22,
  23,
  4,
  23,
  5,
  10,
  12,
  24,
  38,
  35,
  37,
  12,
  16,
  21,
  6,
  0,
  28,
  38,
  6,
  15,
  12,
  16,
  21,
  35,
  29,
  21,
  35,
  22,
  15,
  22,
  7,
  0],
 'chunk_tags': [0,
  11,
  11,
  12,
  12,
  0,
  11,
  0,
  0,
  11,
  12,
  21,
  22,
  22,
  11,
  12,
  12,
  0,
  0,
  11,
  21,
  0,
  13,
  11,
  12,
  12,
  13,
  11,
  12,
  13,
  11,
  13,
  11,
  12,
  0],
 'ner_tags': [0,
  0,
  0,
  7,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  5,
  0

In [16]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [17]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [18]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 '"',
 'neither',
 'the',
 'national',
 'socialists',
 '(',
 'nazis',
 ')',
 'nor',
 'the',
 'communists',
 'dared',
 'to',
 'kidnap',
 'an',
 'american',
 'citizen',
 ',',
 '"',
 'he',
 'shouted',
 ',',
 'in',
 'an',
 'oblique',
 'reference',
 'to',
 'his',
 'extra',
 '##dition',
 'to',
 'germany',
 'from',
 'denmark',
 '.',
 '"',
 '[SEP]']

In [19]:
tokenized_input.word_ids()

[None,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 None]

In [20]:
def tokenize_and_preserve_labels(sentence):
    tokenized_sentence = tokenizer(sentence["tokens"], is_split_into_words=True)
    labels = []
    for word_idx in tokenized_sentence.word_ids():
        label = tokenized_input.word_ids
        if word_idx is None:
            labels.append(-100)
        else:
            labels.append(sentence["ner_tags"][word_idx])
    tokenized_sentence["labels"] = labels
    return tokenized_sentence

In [21]:
sentence = dataset["train"][0]
tokenized_sentence = tokenize_and_preserve_labels(sentence)
print(tokenized_sentence)
# print([dataset["train"].features[f"ner_tags"].feature.names[label] for label in tokenized_sentence.labels if label >= 0])

{'input_ids': [101, 1000, 4445, 1996, 2120, 21633, 1006, 13157, 1007, 4496, 1996, 13009, 15048, 2000, 22590, 2019, 2137, 6926, 1010, 1000, 2002, 6626, 1010, 1999, 2019, 20658, 4431, 2000, 2010, 4469, 20562, 2000, 2762, 2013, 5842, 1012, 1000, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 7, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 5, 0, 0, -100]}


In [22]:
tokenized_dataset = dataset.map(tokenize_and_preserve_labels)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [23]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
import evaluate
seqeval = evaluate.load("seqeval")

In [26]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
id2label = dict(zip(range(len(label_list)), label_list))
label2id = {v: k for k, v in id2label.items()}

In [29]:
import torch
from transformers import BertTokenizer, BertForTokenClassification, TrainingArguments, Trainer
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(label_list), id2label=id2label, label2id=label2id)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args = TrainingArguments(
    output_dir="ner-with-bert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`